# Building contrastive examples

In [1]:
import pandas as pd

df = pd.read_csv('../../data/raw/hyperbook.csv', index_col='url')

In [5]:
import spacy

nlp = spacy.load('en_core_sci_md')  # This can take a while

In [6]:
import re

FIG_RE = re.compile(r"^(fig(ure)?|table|eq(uation)?)\s*\d+", re.I)
CIT_RE = re.compile(r"^\[?\d{1,3}\]?$")

def is_good(sent: str) -> bool:
    """
    Validates a sentence and returns a boolean.
    Excludes figures, citations, section heads, and
    sentences with alphabetic ratios less than 40%,
    less than 4 tokens or more than 128 tokens.
    """
    text = sent.strip()
    # Exclude figures and citations
    if FIG_RE.match(text) or CIT_RE.match(text): return False
    # Exclude section heads
    if text.isupper() and len(text.split()) <= 6: return False
    doc = nlp(text)
    if len(doc) < 4 or len(doc) > 128: return False
    # Ensure alphabetic ratio greater than 40%
    alpha_ratio = sum(t.is_alpha for t in doc)/len(doc) 
    return alpha_ratio >= 0.4

def clean_paper(content):
    doc = nlp(content)
    for sent in doc.sents:
        if is_good(sent.text):
            yield sent.text.strip()

In [11]:
from tqdm import tqdm

sentences, paper_ids, positives = [], [], []
output_path = '../../data/processed/triples.json'

papers = list(df['content'])

for pid, content in tqdm(list(enumerate(papers))):
    sents = list(clean_paper(content))
    # Last sentence has no positive
    for i in range(len(sents) - 1):
        sentences.append(sents[i])
        positives.append(sents[i + 1])
        paper_ids.append(pid)

100%|██████████| 207/207 [10:39<00:00,  3.09s/it]


In [12]:
import json

# Save the data — it takes a while to compute!
data = {'sentences': sentences, 'positives': positives, 'paper_ids': paper_ids}
output_path = '../../data/processed/triples.json'

with open(output_path, 'w') as file:
    json.dump(data, file)

## Hard-negative mining for InfoNCE

In [13]:
from sentence_transformers import SentenceTransformer

model_name = 'Qwen/Qwen3-Embedding-0.6B'
model = SentenceTransformer(model_name)

In [16]:
embeddings = model.encode(
    sentences, 
    batch_size=64, 
    normalize_embeddings=True,  # Normalizes into unit vectors
    show_progress_bar=True
)

Batches:   0%|          | 0/800 [00:00<?, ?it/s]

In [18]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)    # Cosine because vectors are unit-norm
index.add(embeddings)

In [19]:
def mine_negatives(i, k=3, lower=0.6, upper=0.9):
    # Get the top 50 candidates
    D, I = index.search(embeddings[i:i+1], 50)
    hard = []

    for j, score in zip(I[0], D[0]):
        if j == i: continue
        if paper_ids[j] == paper_ids[i]: continue
        if lower <= score <= upper: hard.append(sentences[j])
        if len(hard) == k: break

    return hard

In [ ]:
import json

output_path = '../../data/processed/hyperbook_infonce.jsonl'

with open(file=output_path, mode='w', encoding='utf-8') as file:
    for i, (anchor, positive) in tqdm(list(enumerate(zip(sentences, positives)))):
        negatives = mine_negatives(i)
        file.write(json.dumps({
            'query': anchor,
            'response': positive,
            'rejected_response': negatives
        }, ensure_ascii=False) + '\n')

100%|██████████| 51146/51146 [12:33<00:00, 67.90it/s]
